#PROBLEM STATEMENT

Create a multiagent system as an Event Planner. You have to plan a AI Tech conference for 1000 people in Bengaluru. Create multiple agents to look for the appropriate venue, handle logistics and plan for promotions of the event.

In [11]:
import requests
from bs4 import BeautifulSoup
import threading
import time


event_plan = {
    "venue": None,
    "logistics": None,
    "promotions": None
}
lock = threading.Lock()


def fetch_venue_data():
    print("Venue Agent: Fetching real-time venue data...")
    url = "https://www.venuelook.com/conference-venues-in-bangalore"
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        venues = []
        for venue in soup.select(".row"):
            try:
                name_element = venue.select_one(".vname")
                capacity_element = venue.select_one(".space-capacity")
                cost_element = venue.select_one(".veg-price-green")

                name = name_element.text.strip() if name_element else "Unnamed Venue"
                capacity_text = capacity_element.text.strip() if capacity_element else "0"
                capacity = int("".join(filter(str.isdigit, capacity_text)))
                cost_text = cost_element.text.strip() if cost_element else "N/A"
                cost = int("".join(filter(str.isdigit, cost_text))) if cost_text != "N/A" else None

                venues.append({"name": name, "capacity": capacity, "cost": cost})
            except Exception as ve:
                print(f"Skipped a venue due to parsing error: {ve}")
                print(f"Debugging skipped venue HTML:\n{venue.prettify()}")

        return venues

    except Exception as e:
        print(f"Error fetching venue data: {e}")
        return []


def venue_agent():
    venues = fetch_venue_data()
    if not venues:
        print("Venue Agent: No data fetched from the website.")
        return

    print("Venue Agent: Evaluating venues...")
    time.sleep(2)  # Simulate processing delay

    # Select the most suitable venue
    selected_venue = next((venue for venue in venues if venue["capacity"] >= 1000), None)

    with lock:
        if selected_venue:
            event_plan["venue"] = selected_venue
            print(f"Venue Agent: Selected {selected_venue['name']} with capacity {selected_venue['capacity']} and cost ₹{selected_venue.get('cost', 'N/A')}.")
        else:
            print("Venue Agent: No suitable venue found.")

# Agent to handle logistics planning
def logistics_agent():
    print("Logistics Agent: Planning logistics...")
    time.sleep(3)  # Simulate planning delay
    logistics = {
        "seating_arrangement": "Theatre style for 1000 attendees",
        "catering": "Buffet (vegetarian and non-vegetarian)",
        "audio_visual": "Stage setup with LED screens and surround sound",
        "transport": "Shuttle service from major hubs"
    }

    with lock:
        event_plan["logistics"] = logistics
        print("Logistics Agent: Finalized logistics plan.")

# Agent to handle promotions planning
def promotions_agent():
    print("Promotions Agent: Planning promotions...")
    time.sleep(4)  # Simulate planning delay
    promotions = {
        "email_campaign": "Targeted emails to 5,000 professionals",
        "social_media": "LinkedIn, Twitter, and Facebook ads",
        "media_partners": ["YourStory", "Analytics India Magazine"]
    }

    with lock:
        event_plan["promotions"] = promotions
        print("Promotions Agent: Finalized promotions plan.")

# Master controller
def main():
    print("Starting Multi-Agent Event Planner...\n")

    # Create threads for agents
    threads = [
        threading.Thread(target=venue_agent),
        threading.Thread(target=logistics_agent),
        threading.Thread(target=promotions_agent)
    ]

    # Start all threads
    for thread in threads:
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    # Display the final event plan
    print("\nEvent Planning Complete. Final Plan:")
    print(f"Venue: {event_plan['venue']}")
    print(f"Logistics: {event_plan['logistics']}")
    print(f"Promotions: {event_plan['promotions']}")

# Execute the program
if __name__ == "__main__":
    main()


Starting Multi-Agent Event Planner...

Venue Agent: Fetching real-time venue data...
Logistics Agent: Planning logistics...
Promotions Agent: Planning promotions...
Venue Agent: Evaluating venues...
Logistics Agent: Finalized logistics plan.
Venue Agent: Selected Tranquil Woods By Jade with capacity 10300 and cost ₹3500.
Promotions Agent: Finalized promotions plan.

Event Planning Complete. Final Plan:
Venue: {'name': 'Tranquil Woods By Jade', 'capacity': 10300, 'cost': 3500}
Logistics: {'seating_arrangement': 'Theatre style for 1000 attendees', 'catering': 'Buffet (vegetarian and non-vegetarian)', 'audio_visual': 'Stage setup with LED screens and surround sound', 'transport': 'Shuttle service from major hubs'}
Promotions: {'email_campaign': 'Targeted emails to 5,000 professionals', 'social_media': 'LinkedIn, Twitter, and Facebook ads', 'media_partners': ['YourStory', 'Analytics India Magazine']}
